In [ ]:
install 

In [1]:
import rioxarray as rxr
import pandas as pd
import numpy as np
import rasterio
import os
from pathlib import Path
import matplotlib.pyplot as plt
import richdem as rd
import xarray as xr


ModuleNotFoundError: No module named 'rioxarray'

In [ ]:
s1_thuongxanh_path = r'../INPUT/S1/S1_thuongxanh.tif'
s1_rungla_path = r'../INPUT/S1/S1_rungla.tif'

s2_thuongxanh_path = r'../INPUT/S2/S2_thuongxanh.tif'
s2_rungla_path = r'../INPUT/S2/S2_rungla.tif'
s2_mean_path = r'../INPUT/S2/S2_2024.tif'

dem_path =  r'../INPUT/DEM/DEM.tif'

In [ ]:
# Đọc dữ liệu Sentinel-1
s1_thuongxanh = rxr.open_rasterio(s1_thuongxanh_path, chunks=True)
s1_rungla = rxr.open_rasterio(s1_rungla_path, chunks=True)

print(f"S1 Thường Xanh shape: {s1_thuongxanh.shape}")
print(f"S1 Rừng Lá shape: {s1_rungla.shape}") 
print("Sentinel-1 data loaded successfully!")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.imshow(s1_thuongxanh[1, :, :])  # Band đầu tiên
plt.title("Sentinel-1 Thường Xanh - Band 1")
plt.colorbar()
plt.show()


In [ ]:
# Đọc dữ liệu Sentinel-2
s2_thuongxanh = rxr.open_rasterio(s2_thuongxanh_path, chunks=True)
s2_rungla = rxr.open_rasterio(s2_rungla_path, chunks=True)

s2_mean = rxr.open_rasterio(s2_mean_path, chunks=True)  # Sửa lỗi chính tả

print(f"S2 Thường Xanh shape: {s2_thuongxanh.shape}")
print(f"S2 Rừng Lá shape: {s2_rungla.shape}")

print(f"S2 mean shape: {s2_mean.shape}")

print("Sentinel-2 data loaded successfully!")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.imshow(s2_thuongxanh[0, :, :])  # Band đầu tiên
plt.title("Sentinel-2 Thường Xanh - Band 1")
plt.colorbar()
plt.show()


In [ ]:
# Đọc dữ liệu DEM
dem_data = rxr.open_rasterio(dem_path, chunks=True)
dem_data = dem_data.rio.reproject_match(s2_mean)

print(f"DEM shape: {dem_data.shape}")
print(f"DEM CRS: {dem_data.rio.crs}")
print(f"DEM bounds: {dem_data.rio.bounds()}")
print("DEM loaded successfully!")

In [ ]:
# Định nghĩa các band cho Sentinel-2
# Band order trong dữ liệu: B2(blue), B3(green), B4(red), B8(nir), B11(swir1), B12(swir2)
# Index (0-based):             0         1       2        3          4          5

def calculate_ndvi(s2_data):
    """
    Tính NDVI = (NIR - Red) / (NIR + Red)
    NIR = band 4 (B8), Red = band 3 (B4)
    """
    nir = s2_data.isel(band=3) / 10000.0  # B8 (NIR) is the 4th band, so index is 3
    red = s2_data.isel(band=2) / 10000.0  # B4 (Red) is the 3rd band, so index is 2
    
    ndvi = (nir - red) / (nir + red)  # Tránh chia cho 0
    return ndvi

def calculate_evi(s2_data):
    """
    Tính EVI = 2.5 * ((NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1))
    """
    nir = s2_data.isel(band=3) / 10000.0   # B8 (NIR) is the 4th band, so index is 3
    red = s2_data.isel(band=2) / 10000.0   # B4 (Red) is the 3rd band, so index is 2
    blue = s2_data.isel(band=0) / 10000.0  # B2 (Blue) is the 1st band, so index is 0
    
    evi = 2.5 * ((nir - red) / (nir + 6*red - 7.5*blue + 1))
    return evi

def copy_structure(ref, ndvi):
    ndvi = ndvi.expand_dims(dim={'band': [1]})
    ndvi = ndvi.assign_coords(band=[1])
    
    ndvi = ndvi.rio.write_crs(ref.rio.crs)
    ndvi = ndvi.rio.set_spatial_dims(x_dim="x", y_dim="y")
    return ndvi



In [ ]:
s2_rungla.band

In [ ]:
# Tính NDVI cho cả hai khu vực
ndvi_rungla = calculate_ndvi(s2_rungla)
ndvi_rungla = copy_structure(s2_rungla, ndvi_rungla)

ndvi_thuongxanh = calculate_ndvi(s2_thuongxanh)
ndvi_thuongxanh = copy_structure(s2_rungla, ndvi_thuongxanh)

ndvi_mean = calculate_ndvi(s2_mean)
ndvi_mean = copy_structure(s2_rungla, ndvi_mean)

evi = calculate_evi(s2_mean)
evi = copy_structure(s2_rungla, evi)

# Tính sự chênh lệch NDVI (Rừng Lá - Thường Xanh)
ndvi_difference = ndvi_thuongxanh - ndvi_rungla
        
print("Đã tính toán và đồng bộ hóa thông tin không gian cho tất cả chỉ số Sentinel-2")

In [ ]:
# res_test = xr.concat([s2_rungla, ndvi_rungla, ndvi_thuongxanh, ndvi_mean, evi, ndvi_difference, s2_thuongxanh, s2_mean, dem_data_aligned], dim='band')
# res_test.shape



In [ ]:
import numpy as np

# Giả sử biến là s1_rungla (kiểu rioxarray hoặc xarray)
data = ndvi_difference.values

# Loại bỏ NaN
valid_values = data[~np.isnan(data)]

# Kiểm tra xem có giá trị hợp lệ không
if valid_values.size > 0:
    min_val = np.min(valid_values)
    max_val = np.max(valid_values)
    print("Giá trị nhỏ nhất:", min_val)
    print("Giá trị lớn nhất:", max_val)
else:
    print("Không có giá trị hợp lệ (tất cả đều NaN).")


# Xử lý dữ liệu Sentinel-1

In [ ]:
#Dư liệu S1 gồm 2 bands: 1(VV), 2(VH)
def cal_texture(s1_data):
    VV = s1_data.sel(band=1)
    VH = s1_data.sel(band=2)

    div = VV/VH
    sub = VV-VH
    return div, sub


In [ ]:
# Tính toán texture cho S1
div_rungla, sub_rungla = cal_texture(s1_rungla)
div_thuongxanh, sub_thuongxanh = cal_texture(s1_thuongxanh)

div_rungla = copy_structure(s1_rungla, div_rungla)
sub_rungla = copy_structure(s1_rungla, sub_rungla)

div_thuongxanh = copy_structure(s1_thuongxanh, div_thuongxanh)
sub_thuongxanh = copy_structure(s1_thuongxanh, sub_thuongxanh)


print("Đã tính toán texture S1 và đồng bộ hóa thông tin không gian")

In [ ]:
res_test = xr.concat([s2_rungla, ndvi_rungla, ndvi_thuongxanh, ndvi_mean, evi, ndvi_difference, s2_thuongxanh, s2_mean, s1_rungla, s1_thuongxanh, div_rungla, sub_rungla, div_thuongxanh, sub_thuongxanh, ], dim='band')
res_test.shape

# Xử lý dữ liệu DEM
=> tính toán các chỉ số Slope, Aspect, Focal_mean, TPI, Hillshade


In [ ]:
dem_data

In [ ]:
# Tính toán các chỉ số địa hình từ DEM
from scipy import ndimage

# Lấy mảng DEM và bảo đảm không còn chiều band
dem_array = dem_data.squeeze().rio.write_nodata(-9999)
dem_np = dem_array.values.astype(np.float32)

nodata = dem_array.rio.nodata
mask = np.zeros_like(dem_np, dtype=bool)
if nodata is not None:
    mask |= dem_np == nodata
mask |= np.isnan(dem_np)

dem_np_clean = np.where(mask, np.nan, dem_np)

# Thiết lập richdem rdarray với thông tin địa lý
pixel_size_x = float(abs(dem_array.x.values[1] - dem_array.x.values[0]))
pixel_size_y = float(abs(dem_array.y.values[1] - dem_array.y.values[0]))

dem_rd = rd.rdarray(np.where(mask, -9999, dem_np), no_data=-9999)
dem_rd.geotransform = (
    float(dem_array.x.values[0]),
    pixel_size_x,
    0.0,
    float(dem_array.y.values[0]),
    0.0,
    -pixel_size_y
)

print("Bắt đầu tính toán các chỉ số địa hình từ DEM...")

# 1. Slope (độ dốc, đơn vị độ)
slope = np.array(rd.TerrainAttribute(dem_rd, attrib='slope_degrees'), dtype=np.float32)
slope[mask] = np.nan
print("  ✓ Slope")

# 2. Aspect (hướng dốc, 0-360 độ)
aspect = np.array(rd.TerrainAttribute(dem_rd, attrib='aspect'), dtype=np.float32)
aspect[(aspect < 0) | mask] = np.nan
print("  ✓ Aspect")

# 3. Focal mean (trung bình cục bộ 3x3)
focal_mean = ndimage.generic_filter(dem_np_clean, np.nanmean, size=3, mode='nearest')
focal_mean[mask] = np.nan
print("  ✓ Focal mean")

# 4. TPI (Topographic Position Index)
tpi = dem_np_clean - focal_mean
tpi[mask] = np.nan
print("  ✓ TPI")

# 5. Hillshade (bóng đổ) sử dụng slope & aspect
azimuth_deg = 315.0
altitude_deg = 45.0
azimuth_rad = np.deg2rad(azimuth_deg)
altitude_rad = np.deg2rad(altitude_deg)

slope_rad = np.deg2rad(slope)
aspect_rad = np.deg2rad(np.where(np.isnan(aspect), 0.0, aspect))

hillshade = (
    np.sin(altitude_rad) * np.sin(slope_rad) +
    np.cos(altitude_rad) * np.cos(slope_rad) * np.cos(azimuth_rad - aspect_rad)
)
hillshade = np.clip(hillshade, 0, None)  # giá trị âm -> 0
hillshade = np.where(mask, np.nan, 255 * hillshade)
print("  ✓ Hillshade")

In [ ]:

print("\nChuyển đổi các chỉ số thành rioxarray DataArray...")

# Hàm copy cấu trúc địa lý (nếu đã định nghĩa trước)
# copy_structure(source, target)

# 1. Slope
slope_da = xr.DataArray(
    slope,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
slope_da = copy_structure(dem_data, slope_da)
print("  ✓ Slope DataArray")

# 2. Aspect
aspect_da = xr.DataArray(
    aspect,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
aspect_da = copy_structure(dem_data, aspect_da)
print("  ✓ Aspect DataArray")

# 3. Focal mean
focal_mean_da = xr.DataArray(
    focal_mean,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
focal_mean_da = copy_structure(dem_data, focal_mean_da)
print("  ✓ Focal mean DataArray")

# 4. TPI
tpi_da = xr.DataArray(
    tpi,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
tpi_da = copy_structure(dem_data, tpi_da)
print("  ✓ TPI DataArray")

# 5. Hillshade
hillshade_da = xr.DataArray(
    hillshade,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
hillshade_da = copy_structure(dem_data, hillshade_da)
print("  ✓ Hillshade DataArray")

# 6. DEM cleaned
dem_clean_da = xr.DataArray(
    dem_np_clean,
    dims=["y", "x"],
    coords={"y": dem_array["y"], "x": dem_array["x"]},
)
dem_clean_da = copy_structure(dem_data, dem_clean_da)
print("  ✓ DEM cleaned DataArray")

print("\n✓ Hoàn thành chuyển đổi tất cả các chỉ số địa hình!")

In [ ]:
final_res = xr.concat([s1_rungla, div_rungla, sub_rungla, s1_thuongxanh, div_thuongxanh, sub_thuongxanh, s2_mean, ndvi_rungla, ndvi_thuongxanh, ndvi_difference, ndvi_mean, evi, dem_data, slope_da, aspect_da, focal_mean_da, tpi_da, hillshade_da], dim='band') 
final_res



In [ ]:
from pathlib import Path

final_res_clean = final_res.copy()
if 'long_name' in final_res_clean.attrs:
    del final_res_clean.attrs['long_name']

output_path = Path('../INPUT/stacked/')
# Lưu file
final_res_clean.rio.to_raster(output_path / 'final_stack.tif')
print("✓ Đã lưu file thành công!")